Dataset: https://www.kaggle.com/datasets/usgs/earthquake-database

Descarga automatica del dataset en Kaggle

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("usgs/earthquake-database")

print("Path to dataset files:", path)

Inicio del proceso

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import io

In [ ]:
df_earthquakes = pd.read_csv(f"{path}/database.csv")
df_earthquakes.columns = df_earthquakes.columns.str.lower().str.replace(' ','_')
df_earthquakes['date'] = pd.to_datetime(df_earthquakes['date'], dayfirst=False, errors='coerce').dt.date
df_earthquakes.loc[df_earthquakes["date"].isna(), 'date'] = pd.to_datetime(df_earthquakes.loc[df_earthquakes["date"].isna(), 'time'], errors='coerce').dt.date
columns = ['date','latitude','longitude','type','depth','magnitude','id','source']
df_earthquakes = df_earthquakes[columns]
df_earthquakes.head(3)


Datos adicionales generados con IA a partir de los acronimos de la columna source de earthquakes

In [ ]:
data = {
    "source": [
        "ISCGEM", "ISCGEMSUP", "OFFICIAL", "CI", "US",
        "NC", "GCMT", "UW", "ATLAS", "NN", "SE", "AK", "PR"
    ],
    "source_name": [
        "Catálogo Global de Sismos ISC-GEM",
        "Catálogo Global de Sismos ISC-GEM",
        "Agencia Oficial Local/Regional",
        "Red Sísmica del Sur de California",
        "Servicio Geológico de EE. UU. (USGS)",
        "Red Sísmica del Norte de California",
        "Global Centroid Moment Tensor",
        "Red Sísmica del Noroeste del Pacífico",
        "Catálogo Sísmico ATLAS",
        "Laboratorio Sismológico de Nevada",
        "Red Sísmica del Sureste de EE. UU.",
        "Centro de Terremotos de Alaska",
        "Red Sísmica de Puerto Rico"
    ],
    "source_description": [
        "Catálogo histórico que re-analiza y estandariza datos sísmicos antiguos.",
        "Catálogo histórico que re-analiza y estandariza datos sísmicos antiguos.",
        "Fuente oficial del país o región donde ocurrió el sismo (genérico).",
        "Colaboración entre USGS y Caltech para monitorear el sur de California.",
        "Agencia principal de monitoreo sísmico a nivel mundial, especialmente para eventos recientes.",
        "Colaboración entre USGS y Berkeley para monitorear el norte de California.",
        "Proyecto que determina los mecanismos focales de sismos de magnitud moderada a grande.",
        "Red regional de la Universidad de Washington para monitorear Washington y Oregón.",
        "Un compendio o catálogo de sismos, como el 'Centennial' o proyectos de re-análisis.",
        "Red sísmica regional que monitorea la actividad en Nevada y áreas circundantes.",
        "Red regional que monitorea la actividad sísmica en el sureste de los Estados Unidos.",
        "Red sísmica regional para el monitoreo del estado de Alaska.",
        "Monitorea la actividad sísmica en la región de Puerto Rico y el Caribe."
    ]
}

df_source = pd.DataFrame(data)
df_source.head(3)


In [ ]:
df_earthquakes = pd.merge(df_earthquakes, df_source, on='source', how='left')
df_earthquakes.head(3)

### CREATE TABLE en Postgresql
```
CREATE TABLE earthquakes (
    date DATE,
    latitude FLOAT,
    longitude FLOAT,
    type VARCHAR(50),
    depth FLOAT,
    magnitude FLOAT,
    id VARCHAR(50),
    source VARCHAR(50),
    source_name VARCHAR(100),
    source_description VARCHAR(200)
);
```




In [ ]:

def insert_data(df,table_name,db_host,db_port,db_name,db_user,db_password):
  try:
      # Crea el motor de conexión con SQLAlchemy
      engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
      conn = engine.raw_connection()
      cur = conn.cursor()
      print("Conexión a PostgreSQL exitosa.")

      # Prepara un "archivo en memoria" para la carga masiva
      buffer = io.StringIO()
      # Escribe el contenido del DataFrame al buffer en formato CSV (sin cabecera ni índice)
      df.to_csv(buffer, index=False, header=False)
      buffer.seek(0) # Regresa al inicio del buffer para la lectura

      # Ejecuta el comando COPY de PostgreSQL para una inserción masiva y eficiente
      print(f"Cargando {len(df)} registros en la tabla '{table_name}'...")
      cur.copy_expert(f"""COPY {table_name} FROM STDIN WITH (FORMAT CSV)""", buffer)

      # Confirma la transacción
      conn.commit()
      print("Todos los registros han sido insertados.")

  except Exception as error:
      print("Error al conectar o insertar en la base de datos:", error)
      if 'conn' in locals() and conn:
          conn.rollback() # Revierte los cambios si hubo un error

  finally:
      # Cierra la conexión
      if 'cur' in locals() and cur:
          cur.close()
      if 'conn' in locals() and conn:
          conn.close()
          print("Conexión a PostgreSQL cerrada.")

En caso de problemas con el Firewall (Reglas de entrada del servidor del Postgresql) se puede obtener la ip del Google Colab con:


```
!curl ipecho.net/plain
```



In [ ]:
!curl ipecho.net/plain

In [ ]:


db_host = "#COMPLETAR#"
db_port = "5432"
db_name = "#COMPLETAR#"
db_user = "#COMPLETAR#"
db_password = "#COMPLETAR#"
table_name = "earthquakes"

insert_data(df_earthquakes,table_name,db_host,db_port,db_name,db_user,db_password)